In [1]:
import numpy as np
import argparse
import imutils #makes basix image processing resizing etc a lot easier
import sys
import cv2

In [2]:
act_lst=open("Actions.txt").read().strip().split("\n")
#getting labels for all possible activities

In [3]:
# loading the kinetic model
md = cv2.dnn.readNet("resnet-34_kinetics.onnx")

In [7]:
cap=cv2.VideoCapture(0)
while True:

    #now since we are video processing we would need continuous set of images instead of single frame by frame images we will need to convert it to blob 
    to_process=[]
    original=[]
    # lets take 16 frames at a time to make blob
    for i in range(0,16):
        bl,img=cap.read()
        cv2.imshow('wnd',img)
        if bl==False:
            print("Unable to get input. Exiting code")
            cv2.destroyAllWindows()
            break
        original.append(img)
        img=imutils.resize(img,400)
        to_process.append(img)
    blob=cv2.dnn.blobFromImages(to_process,1.0,(112,112), (114.7748, 107.7354, 99.4750),swapRB=True, crop=True)
    blob = np.transpose(blob, (1, 0, 2, 3))
    
    blob = np.expand_dims(blob, axis=0)
    print(blob.shape)
    md.setInput(blob)
    ot=md.forward()
    label=act_lst[np.argmax(ot)]

    for frame in original:
        # append predicted activity
        
        cv2.rectangle(frame, (0, 0), (300, 40),
                      (0, 0, 0), -1)
        cv2.putText(frame, label, (10, 25),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                    (255, 255, 255), 2)
        cv2.imshow('wnd',frame)
    if cv2.waitKey(1) & 0XFF==ord('a'):
        break
cv2.destroyAllWindows()

(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 112, 112)
(1, 3, 16, 11

KeyboardInterrupt: 